In [1]:
import requests
import time
import datetime
import schedule
import os
import json
import newspaper

In [2]:
def send_alert_message():
    return requests.post(
        "https://api.mailgun.net/v3/sandboxba85a9310ad64d3998761e86206d868f.mailgun.org/messages",
        auth=("api", "key-5e11abc6288b38c6dcaeb2934f835a42"),
        data={"from": "Mailgun Sandbox <postmaster@sandboxba85a9310ad64d3998761e86206d868f.mailgun.org>",
              "to": "Brennan Borlaug <borlaug13@gmail.com>",
              "subject": "Real News Extract Script Halted",
              "text": "credible_sources_extract.ipynb is no longer running on ec2-instance"})

In [3]:
urls=['https://newsapi.org/v1/articles?source=independent&sortBy=top&apiKey=69ba1408fe9b4ffb821fe13f4cf56244',
      'https://newsapi.org/v1/articles?source=bbc-news&sortBy=top&apiKey=69ba1408fe9b4ffb821fe13f4cf56244',
      'https://newsapi.org/v1/articles?source=reuters&sortBy=top&apiKey=69ba1408fe9b4ffb821fe13f4cf56244',
      'https://newsapi.org/v1/articles?source=the-wall-street-journal&sortBy=top&apiKey=69ba1408fe9b4ffb821fe13f4cf56244',
      'https://newsapi.org/v1/articles?source=the-washington-post&sortBy=top&apiKey=69ba1408fe9b4ffb821fe13f4cf56244',
      'https://newsapi.org/v1/articles?source=the-new-york-times&sortBy=top&apiKey=69ba1408fe9b4ffb821fe13f4cf56244',
      'https://newsapi.org/v1/articles?source=national-geographic&sortBy=top&apiKey=69ba1408fe9b4ffb821fe13f4cf56244',
      'https://newsapi.org/v1/articles?source=usa-today&sortBy=top&apiKey=69ba1408fe9b4ffb821fe13f4cf56244']

In [4]:
def extract_articles():
    print('Extracting...')
    for url in urls:
        r = requests.get(url)
        source = url.replace('https://newsapi.org/v1/articles?source=','').replace('&sortBy=top&apiKey=69ba1408fe9b4ffb821fe13f4cf56244','')
        print('--{}...'.format(source))
        filename = '/ebs_volume/data/Credible/{0}/articles/{1}/api.txt'.format(source, date)
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        
        with open(filename, 'w') as f:
            json.dump(json.loads(r.text), f)
            
        with open(filename) as f:
                jdata = json.load(f)
                jdata2=jdata['articles']

        for i in range(0,5):
            r=jdata2[i]['url']
            link = newspaper.Article(r)
            link.download()
            html = link.html
            if link.is_downloaded:
                link.parse()
                date_longform = '{0}-2017'.format(date)
                article = {}
                article["html"] = html
                article["title"] = link.title
                article["url"] = link.url
                article["date"] = date_longform
                article["source"] = source
                article["text"] = link.text
                article["images"] = list(link.images)
                article["videos"] = link.movies
                count=i+1
                filename = '/ebs_volume/data/Credible/{0}/articles/{1}/article_{2}.txt'.format(source, date, count)
                os.makedirs(os.path.dirname(filename), exist_ok=True)
                with open(filename, 'w',encoding="utf8",newline='') as file:
                    json.dump(article, file)
            
    print('Finished')
    return None

In [ ]:
schedule.every().day.at("4:00").do(extract_articles)
while True:
    try:
        date = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%m-%d") #accounts for UTC
        schedule.run_pending()
        time.sleep(60)
    except:
        send_alert_message()
        print("Error...")
        break

Extracting...
--independent...
--bbc-news...
--reuters...
--the-wall-street-journal...
--the-washington-post...
--the-new-york-times...
--national-geographic...
--usa-today...
Finished
